# Εργασία 3: Παρεμβολή, παλινδρόμηση, εύρεση ριζών

## Οδηγίες

- Όι ζητούμενες συναρτήσεις δεν πρέπει έχουν αναφορές σε global μεταβλητές εκτός από σταθερές καθιερωμένων βιβλιοθηκών (π.χ numpy.pi).
- Ο αριθμός μητρώου πρέπει να είναι σύμφωνος με αυτόν που έχει δηλωθεί στο προφίλ του eclass.
- Όπου χρησιμοποιούνται ψηφία του αριθμού μητρώου, πρέπει να είναι απευθείας δηλωμένα στον κώδικα (hard-coded) και όχι να εξάγονται από τον ΑΜ.
- Το αρχείο της εργασίας πρέπει να τρέχει εξ ολοκλήρου (Run All) για να βαθμολογηθεί.


In [ ]:
# Εισαγωγή μονάδων κώδικα
# - Μην αλλάζετε αυτό το κελί.

import numpy as np
import matplotlib.pyplot as plt


In [10]:
# Στοιχεία φοιτητή
# - Δηλώστε στις παρακάτω μεταβλητές το όνομά σας με ελληνικούς χαρακτήρες και το ΑΜ ως αριθμό.

onoma = 'ΜΑΡΚΟΣ'
eponymo = 'ΠΑΣΣΙΑΣ'
AM = 3124064


## Άσκηση 1

Σε ένα πανεπιστημιακό μάθημα καταγράφηκε η συμμετοχή των φοιτητών στις δια ζώσης διαλέξεις και προέκυψε η παρακάτω χρονοσειρά:

| Διάλεξη | Φοιτητές |
|--|--|
|1|62|
|2|59|
|3|54|
|4|53|
|5|28|
|6|30|
|7|41+a-b|
|8|30|
|9|33|
|10|21+b-c|
|11|30|
|12|28+c-a|
|15|23|
|16|28|
|17|26|
|18|24|

Στις διαλέξεις 13 και 14 δεν έγινε καταμέτρηση των φοιτητών.

Γράψτε μια συνάρτηση `myfunction1` που δέχεται ως παράμετρο την τιμή της διάλεξης και εκτιμά το πλήθος των φοιτητών με:

1. γραμμική παρεμβολή
2. παρεμβολή φυσικών κυβικών spline,
4. παλινδρόμηση πολυώνυμου 3ου βαθμού.
4. παλινδρόμηση πολυώνυμου 6ου βαθμού.

Η συνάρτηση πρέπει να επιστρέφει έναν πίνακα με τις τιμές από τις 4 μεθόδους (πραγματικοί αριθμοί χωρίς στρογγυλοποίηση). Σε τιμές εκτός του πεδίου τιμών $x$ πρέπει να εκτελεί προεκβολή, εκτός από την μέθοδο 1 στην οποία θα χρησιμοποιεί την ακραία τιμή του πίνακα $y$.
Χρησιμοποιώντας την συνάρτηση εκτιμείστε την συμμετοχή των φοιτητών στις διαλέξεις 13, 14 και 19.

In [19]:
def myfunction1(dialexi: np.float64) -> np.ndarray:
    # Συμπληρώστε τις εντολές import εδώ σε περίπτωση που χρειαστείτε επιπλέον μονάδες κώδικα
    from scipy.interpolate import CubicSpline
    import numpy as np
    import numpy.polynomial.polynomial as poly

    x = np.array([0,1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 16, 17, 18])

    a, b, c = 0, 6, 4
    y = np.array([62, 59, 54, 53, 28, 30, 41 + a - b, 30, 33, 21 + b - c, 30, 28 + c - a, 23, 28, 26, 24])

    # 1. Γραμμική παρεμβολή με χρήση ακραίων τιμών για εκτός πεδίου
    if dialexi < x[0]:
        linear_interp = float(y[0])
    elif dialexi > x[-1]:
        linear_interp = float(y[-1])
    else:
        linear_interp = np.interp(dialexi, x, y)

    # 2. Παρεμβολή φυσικών κυβικών spline
    cs = CubicSpline(x, y, bc_type='natural')
    cubic_interp = cs(dialexi)

    # 3. Παλινδρόμηση πολυώνυμου 3ου βαθμού
    coeff3 = poly.polyfit(x, y, 3)
    poly3_interp = poly.polyval(dialexi, coeff3)

    # 4. Παλινδρόμηση πολυώνυμου 6ου βαθμού
    coeff6 = poly.polyfit(x, y, 6)
    poly6_interp = poly.polyval(dialexi, coeff6)

    return np.array([linear_interp, cubic_interp, poly3_interp, poly6_interp])


In [ ]:
# Πρόχειρο άσκησης 1
# - Σε αυτό το κελί μπορείτε να τυπώσετε μεταβλητές και να κάνετε γραφικές παραστάσεις για επαλήθευση. Δεν λαμβάνεται υπόψη στην βαθμολόγηση.

print(myfunction1(13),myfunction1(14),myfunction1(19))

## Άσκηση 2

Δίνεται η παρακάτω εξίσωση που εκφράζει το ισοζύγιο ισχύος σε ένα όχημα που κινείται με σταθερή ταχύτητα σε επίπεδο οδόστρωμα.

$$
n_{tr}\dot{W}_{b}=(C_DA_f \frac{1}{2}\rho V^2+C_Rmg)V
$$

όπου:
- $n_{tr}=0.95$, o βαθμός απόδοσης του συστήματος μετάδοσης κίνησης
- $\dot{W}_{b}\; [\text{W}]$, η αποδιδόμενη ισχύς από τον κινητήρα
- $C_D=0.31+(a-b)/100$, ο αεροδυναμικός συντελεστής του οχήματος
- $A_f=2\; m^2$, η μετωπική επιφάνεια του οχήματος
- $\rho=1.2\; \text{kg}/\text{m}^3$,  η πυκνότητα του αέρα
- $V\; [\text{m}/\text{s}]$,  η ταχύτητα του οχήματος
- $C_R =0.015$, η αντίσταση κύλησης
- $m =1000+(b-c)\cdot 10\;\text{kg}$, η μάζα του οχήματος
- $g =9.81\;\text{m}/\text{s}^2$, η επιτάχυνση της βαρύτητας

και a,b,c τα 3 τελευταία ψηφία του αριθμού μητρώου σας (c=τελευταίο).

Γράψτε μια συνάρτηση `myfunction2` που δέχεται ως παράμετρο την ισχύ του κινητήρα $\dot{W}_{b}$ και υπολογίζει την τιμή της ταχύτητας $V$ με απόλυτη ακρίβεια $10^{-3}$ χρησιμοποιώντας τις μεθόδους:

1. διχοτόμησης με όρια $0$ και $100\;\text{m}/\text{s}$.
2. Newton-Raphson με αρχική τιμή $20\;\text{m}/\text{s}$.
3. τέμνουσας με αρχικές τιμές $20$ και $21\;\text{m}/\text{s}$.

Η συνάρτηση πρέπει να επιστρέφει πίνακα της NumPy με τις τρεις τιμές.

Ελέγξτε τα αποτελέσματα της συνάρτησης για τις τιμές ισχύος:

- 5 kW
- 10 kW
- 50 kW

Προσοχή: Να γίνουν οι κατάλληλες μετατροπές μονάδων.


In [20]:
# Λύση της άσκησης 2
# - Σε αυτό το κελί γράψτε μόνο τις ζητούμενες συναρτήσεις χωρίς print και input.
import numpy as np

def myfunction2(Wb: np.float64) -> np.ndarray:

    # Παράμετροι
    ntr = 0.95
    Aj = 2  # m^2
    rho = 1.2  # kg/m^3
    CR = 0.015
    g = 9.81  # m/s^2
    a, b, c = 0, 6, 4
    CD = 0.31 + (a - b) / 100
    m = 1000 + (b - c) * 10

    # Εξίσωση ισοζυγίου ισχύος
    def power_balance(V):
        return ntr * Wb - (CD * Aj * rho * V**3 + CR * m * g * V)

    # Παράγωγος της εξίσωσης ισοζυγίου ισχύος
    def power_balance_derivative(V):
        return - (3 * CD * Aj * rho * V**2 + CR * m * g)

    # 1. Μέθοδος Διχοτόμησης
    def bisection(a_val, b_val, tol=1e-3, max_iter=100):
        if power_balance(a_val) * power_balance(b_val) >= 0:

            if power_balance(a_val) == 0: return a_val
            if power_balance(b_val) == 0: return b_val
            return np.nan

        for _ in range(max_iter):
            c_val = (a_val + b_val) / 2
            if abs(power_balance(c_val)) < tol:
                return c_val
            if power_balance(a_val) * power_balance(c_val) < 0:
                b_val = c_val
            else:
                a_val = c_val
        return (a_val + b_val) / 2

    # 2. Μέθοδος Newton-Raphson
    def newton_raphson(x0, tol=1e-3, max_iter=100):
        x = x0
        for _ in range(max_iter):
            f_x = power_balance(x)
            f_prime_x = power_balance_derivative(x)
            if abs(f_prime_x) < 1e-10:
                return np.nan
            delta_x = f_x / f_prime_x
            x_next = x - delta_x
            if abs(x_next - x) < tol:
                return x_next
            x = x_next
        return x

    # 3. Μέθοδος Τέμνουσας
    def secant(x0, x1, tol=1e-3, max_iter=100):
        x_prev = x0
        x_curr = x1
        for _ in range(max_iter):
            f_prev = power_balance(x_prev)
            f_curr = power_balance(x_curr)
            if abs(f_curr - f_prev) < 1e-10:
                return np.nan
            x_next = x_curr - f_curr * (x_curr - x_prev) / (f_curr - f_prev)
            if abs(x_next - x_curr) < tol:
                return x_next
            x_prev, x_curr = x_curr, x_next
        return x_curr

    return np.array([v_bisection, v_newton_raphson, v_secant])


In [21]:
# Πρόχειρο άσκησης 2
# - Σε αυτό το κελί μπορείτε να τυπώσετε μεταβλητές και να κάνετε γραφικές παραστάσεις για επαλήθευση. Δεν λαμβάνεται υπόψη στην βαθμολόγηση.

print(myfunction2(5e3),myfunction2(10e3),myfunction2(50e3)

SyntaxError: incomplete input (ipython-input-1682858856.py, line 4)